In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [23]:
#%cd 'local_code'
!pwd
#!pip install nilearn
#!pip install pandas --upgrade
%run 'load_data.ipynb'

import pandas as pd
pd.__version__

/content/drive/My Drive/brain project/local_code
/content/drive/My Drive/brain project/local_code


'0.25.0'

In [0]:
data = create_subject_experiment_data(['../data_social_anxiety/social_anxiety_sagol/questionnaires_byTasks_new.xlsx'], 
                                      [('../data_social_anxiety/social_anxiety_sagol/Hariri_2ndLev/FacesVsShapes', 'hariri')])

In [0]:
x,y,z = data[0].tasks_data['hariri'].shape

In [0]:
#this is a small part of the brain images with FPES as y label
hariri_data = [
    {
        'data': subject_data.tasks_data['hariri'].reshape(x*y*z),
        'y': subject_data.features_data['FPES']
    } for subject_data in data if 'hariri' in subject_data.tasks_data]

In [0]:
from sklearn.svm import SVR
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

#split data to X and y
hariri_data_X = [data['data'] for data in hariri_data]
hariri_data_y = [data['y'] for data in hariri_data]

In [29]:
#split data to train and test
X_train, X_test, y_train, y_test = train_test_split(hariri_data_X, hariri_data_y, test_size=0.25)

#train a linear svr
svr = SVR(kernel='linear')
svr.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [30]:
#predict using linear svr
preds = svr.predict(X_test)
print(preds)
print(y_test)

[27.77018722 37.68288065 26.38827449 29.97338336 28.47557469 37.46435092
 34.10675192 29.69034286 30.62332953 40.07009318 31.80459742 24.6712624
 19.63693163 39.87633445]
[26, 23, 23, 37, 25, 33, 48, 29, 45, 12, 55, 28, 19, 20]


In [0]:

# Here I try to implement the spaceNet thing on our 4 samples.
# it is not working since it is not enough samples
# thats the code they used: https://nilearn.github.io/modules/generated/nilearn.decoding.SpaceNetRegressor.html#nilearn.decoding.SpaceNetRegressor


n_subjects = 4  # increase this number if you have more RAM on your box
imgs_paths = ['sub-120con_0001.nii', 'sub-121con_0001.nii', 'sub-122con_0001.nii', 'sub-123con_0001.nii']

# Split data into training set and test set
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split

labels = Y
rng = check_random_state(42)
gm_imgs_train, gm_imgs_test, age_train, age_test = train_test_split(
    imgs_paths, labels, train_size=.75, random_state=rng)

print(gm_imgs_train)
print(gm_imgs_test)



In [0]:
from nilearn.decoding import SpaceNetRegressor

# To save time (because these are anat images with many voxels), we include
# only the 5-percent voxels most correlated with the age variable to fit.
# Also, we set memory_level=2 so that more of the intermediate computations
# are cached. Also, you may pass and n_jobs=<some_high_value> to the
# SpaceNetRegressor class, to take advantage of a multi-core system.
#
# Also, here we use a graph-net penalty but more beautiful results can be
# obtained using the TV-l1 penalty, at the expense of longer runtimes.
decoder = SpaceNetRegressor(memory="nilearn_cache", penalty="graph-net",
                            screening_percentile=5., memory_level=2)
decoder.fit(gm_imgs_train, age_train)  # fit
coef_img = decoder.coef_img_
y_pred = decoder.predict(gm_imgs_test).ravel()  # predict
mse = np.mean(np.abs(age_test - y_pred))
print('Mean square error (MSE) on the predicted age: %.2f' % mse)